In [5]:
import math
def divisors(n):
    divs = [1]
    for i in range(2,int(math.sqrt(n))+1):
        if n%i == 0:
            divs.extend([i,n//i])
    divs.extend([n])
    return list(set(divs))

In [16]:
def switch(n):
    if n==0:
        return 1
    else: 
        return 0

In [25]:
divisors(1)

[1]

In [26]:
l = [0,0,1,0,1,1,0,1,0,0]

In [36]:
#l = [0,0,1,0,1,1,0,1,0,0]
def f(l):
    new_l =[0]*len(l)
    for i in range(len(l),0,-1):
        if l[i-1]==0:
            new_l[i-1]=1
            div = divisors(i)
            for j in div:
                l[j-1]=switch(l[j-1])
        else:
            continue
    return new_l  

In [37]:
file = [int(i) for i in open('100.txt', 'r').read().split()]

In [38]:
f(file)

[0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1]